<a href="https://colab.research.google.com/github/golf41101/Suchet/blob/master/Recomendation_System_phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomendation System - phase1

##Objectives

##Suggest items or some stuff

Situations 

  1 User- User : Users don't have a favorite movie or want to watch or buy something like someone they follow.
  
  2 Item- Item : Users have favorite movie.

###Method : User-Based and Item-Based




---



In [1]:
import pandas as pd
import seaborn as sns
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', 20)
# np.set_printoptions(threshold=sys.maxsize)

# Load Data

In [3]:
from google.colab import drive
#mount you GoogleDrive to "drive" folder
drive.mount('/drive', force_remount=False)

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [4]:
#set path
path ="/drive/MyDrive/Recommendation/"
os.chdir(path)

In [5]:
!ls

movie.csv  rating.csv  ratings_small.csv


In [6]:
rating = pd.read_csv(path + "ratings_small.csv")

In [7]:
movie = pd.read_csv(path + "movie.csv")

In [8]:
movie.drop(columns='genres',inplace=True)

# EDA

In [9]:
df = rating.merge(movie,on='movieId',how='inner')

In [10]:
df.drop(columns='timestamp',inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99343 entries, 0 to 99342
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   99343 non-null  int64  
 1   movieId  99343 non-null  int64  
 2   rating   99343 non-null  float64
 3   title    99343 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


In [12]:
df.isnull().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [13]:
df.head()

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)


In [14]:
# check unique for each columns
for i in df.columns:
  print(df[i].nunique())

671
8794
10
8792


In [15]:
#check duplicate movieId and title, 1 movieId should be 1 title
df_check = df[['movieId','title']]
df_check.drop_duplicates(subset='movieId',inplace=True)

df_check[df_check.duplicated(subset=['title'],keep=False)==True].sort_values('movieId')

,movieId,title
55118,3598,Hamlet (2000)
62452,34048,War of the Worlds (2005)
99338,64997,War of the Worlds (2005)
95304,65665,Hamlet (2000)


In [16]:
#I keep movieId 3598 and 34048 because(order by ascending)
df = df[~df['movieId'].isin([3598,34048])]
df.shape

(99297, 4)

In [17]:
# check again
# check unique for each columns
for i in df.columns:
  print(i,'have',df[i].nunique())

userId have 671
movieId have 8792
rating have 10
title have 8792


# Recommend Movies : User-Based Filtering 

In [18]:
rating_small = df.copy()

In [19]:
rating_small

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)
3,32,31,4.0,Dangerous Minds (1995)
4,36,31,3.0,Dangerous Minds (1995)
...,...,...,...,...
99338,664,64997,2.5,War of the Worlds (2005)
99339,664,72380,3.5,"Box, The (2009)"
99340,665,129,3.0,Pie in the Sky (1996)
99341,665,4736,1.0,Summer Catch (2001)


In [20]:
rating_small[(rating_small['userId']==5) & (rating_small['title']=='As Good as It Gets (1997)')]

,userId,movieId,rating,title
25695,5,1784,4.5,As Good as It Gets (1997)


## create pivot table for filling cosine similarty

In [21]:
rating_mt = rating_small.pivot_table(index='userId', columns='title', values='rating').fillna(0)
print(rating_mt.shape)
rating_mt

(671, 8792)


title,$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (1964),Zulu (2013),[REC] (2007),"\\""Great Performances\""\"" Cats (1998)""",eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# alternate similarity : pearson correlation
user_similarity = rating_mt.T
user_similarity_pe = rating_mt.T.corr()
print(user_similarity_pe.shape)
user_similarity_pe.head()

(671, 671)


userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.004088,-0.003382,0.068592,0.012130,-0.003000,0.079975,-0.005121,0.009729,-0.003184,...,-0.003569,-0.001275,0.003906,0.034915,-0.002887,-0.003890,-0.002046,0.060268,-0.002564,0.012486
2,-0.004088,1.000000,0.118387,0.107040,0.095056,-0.006000,0.206081,0.104139,0.107757,0.037129,...,0.473607,0.114344,0.058718,0.148962,0.463363,0.420996,0.080986,0.018613,0.166438,0.104144
3,-0.003382,0.118387,1.000000,0.071530,0.144867,0.056031,0.148613,0.243608,0.129954,0.110009,...,0.156254,0.116474,0.164770,0.146199,0.173178,0.118936,0.122003,0.076676,0.132965,0.163302
4,0.068592,0.107040,0.071530,1.000000,0.116884,0.070734,0.310253,0.177353,0.020320,0.128474,...,0.104086,0.083729,0.106217,0.230059,0.113918,0.077115,0.062513,0.096248,0.046678,0.198347
5,0.012130,0.095056,0.144867,0.116884,1.000000,0.057251,0.086573,0.155687,0.079900,0.025157,...,0.184321,0.036170,0.126333,0.207628,0.134014,0.049662,0.038407,0.031989,0.057075,0.215797


In [23]:
user_similarity_co = cosine_similarity(rating_mt)
print(user_similarity_co.shape)
print(type(user_similarity_co))

(671, 671)
<class 'numpy.ndarray'>


In [24]:
user_similarity_co

array([[1.        , 0.        , 0.        , ..., 0.06291708, 0.        ,
        0.01746565],
       [0.        , 1.        , 0.12429498, ..., 0.02413984, 0.17059464,
        0.1131753 ],
       [0.        , 0.12429498, 1.        , ..., 0.08098382, 0.13660585,
        0.17019275],
       ...,
       [0.06291708, 0.02413984, 0.08098382, ..., 1.        , 0.04260878,
        0.08520194],
       [0.        , 0.17059464, 0.13660585, ..., 0.04260878, 1.        ,
        0.22867673],
       [0.01746565, 0.1131753 , 0.17019275, ..., 0.08520194, 0.22867673,
        1.        ]])

In [25]:
# fill numpy array data to dataframe
user_sim = pd.DataFrame(user_similarity_co, columns=rating_mt.T.columns,index=rating_mt.T.columns)

In [26]:
user_sim

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014575,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.116373,0.078291,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.118207,0.177371,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.086959,0.137539,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
5,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.038929,0.147200,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.000000,0.425462,0.124562,0.088735,0.058252,0.000000,0.232051,0.069005,0.066412,0.032653,...,0.342283,0.093452,0.074601,0.124214,0.351207,1.000000,0.091597,0.018416,0.153111,0.127995
668,0.000000,0.084646,0.124911,0.068483,0.042926,0.019563,0.058773,0.112366,0.194493,0.098561,...,0.074089,0.108654,0.093675,0.082525,0.114487,0.091597,1.000000,0.000000,0.178017,0.135387
669,0.062917,0.024140,0.080984,0.104309,0.038358,0.024583,0.073151,0.055143,0.029291,0.060549,...,0.015960,0.047787,0.078475,0.101707,0.028773,0.018416,0.000000,1.000000,0.042609,0.085202


## Ready to recommend movies to user by user-user

In [27]:
# user_sim.reset_index(inplace=True)
# user_sim = user_sim.rename_axis(None, axis=1)

In [28]:
user_sim

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014575,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.116373,0.078291,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.118207,0.177371,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.086959,0.137539,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
5,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.038929,0.147200,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.000000,0.425462,0.124562,0.088735,0.058252,0.000000,0.232051,0.069005,0.066412,0.032653,...,0.342283,0.093452,0.074601,0.124214,0.351207,1.000000,0.091597,0.018416,0.153111,0.127995
668,0.000000,0.084646,0.124911,0.068483,0.042926,0.019563,0.058773,0.112366,0.194493,0.098561,...,0.074089,0.108654,0.093675,0.082525,0.114487,0.091597,1.000000,0.000000,0.178017,0.135387
669,0.062917,0.024140,0.080984,0.104309,0.038358,0.024583,0.073151,0.055143,0.029291,0.060549,...,0.015960,0.047787,0.078475,0.101707,0.028773,0.018416,0.000000,1.000000,0.042609,0.085202


In [29]:
## Example recomed userId 5
rec_userId = 5 

## remove userId that picked
# user_sim_rec = user_sim[user_sim['userId']!=rec_userId]
user_sim_rec = user_sim.copy()
# user_sim_rec.drop(index=rec_userId, inplace=True)

In [30]:
user_sim_rec

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014575,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.116373,0.078291,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.118207,0.177371,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.086959,0.137539,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
5,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.038929,0.147200,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.000000,0.425462,0.124562,0.088735,0.058252,0.000000,0.232051,0.069005,0.066412,0.032653,...,0.342283,0.093452,0.074601,0.124214,0.351207,1.000000,0.091597,0.018416,0.153111,0.127995
668,0.000000,0.084646,0.124911,0.068483,0.042926,0.019563,0.058773,0.112366,0.194493,0.098561,...,0.074089,0.108654,0.093675,0.082525,0.114487,0.091597,1.000000,0.000000,0.178017,0.135387
669,0.062917,0.024140,0.080984,0.104309,0.038358,0.024583,0.073151,0.055143,0.029291,0.060549,...,0.015960,0.047787,0.078475,0.101707,0.028773,0.018416,0.000000,1.000000,0.042609,0.085202


In [31]:
user_sim_rec.drop(index=rec_userId, inplace=True)

In [32]:
user_sim_rec.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014575,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.116373,0.078291,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.118207,0.177371,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.086959,0.137539,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
6,0.000000,0.000000,0.060691,0.079648,0.063796,1.000000,0.000000,0.128502,0.021745,0.045264,...,0.012962,0.016633,0.125352,0.082602,0.000000,0.000000,0.019563,0.024583,0.019465,0.087705


In [33]:
# Number of similar users
n = 5

# Get top n similar users
sim_users = user_sim_rec[rec_userId].sort_values(ascending=False)[:n]

# Print out top n similar users
print(f'The similar users for user {rec_userId} are', sim_users)

The similar users for user 5 are userId
313    0.338180
500    0.336893
292    0.320923
442    0.312862
654    0.310267
Name: 5, dtype: float64


In [34]:
# Prepare data by replacing 0 with nan because I want to calculate the rating of movies that have been rated.
rating_mt.replace(0,np.nan,inplace=True)
# rec_userid_watched = rating_mt[rating_mt.index == rec_userId].dropna(axis=1, how='all')

In [35]:
# check results after replace values
j = []
for i in rating_mt.columns:
  print(rating_mt[i].value_counts())
  j.append(i)
  if len(j) == 3:
    break

4.5    2
2.5    1
Name: $9.99 (2008), dtype: int64
2.0    1
Name: 'Hellboy': The Seeds of Creation (2004), dtype: int64
0.5    1
Name: 'Neath the Arizona Skies (1934), dtype: int64


In [36]:
# Movies that the target user has watched
rec_userid_watched = rating_mt[rating_mt.index == rec_userId].dropna(axis=1, how='all')
rec_userid_watched

title,28 Days Later (2002),"40-Year-Old Virgin, The (2005)",Ace Ventura: Pet Detective (1994),Aladdin (1992),Almost Famous (2000),American Pie (1999),American Pie 2 (2001),Antz (1998),Apollo 13 (1995),As Good as It Gets (1997),...,Toy Story 2 (1999),"Truman Show, The (1998)",Vertigo (1958),Walk the Line (2005),Wedding Crashers (2005),"Wedding Singer, The (1998)",What Women Want (2000),When Harry Met Sally... (1989),"Wizard of Oz, The (1939)",You've Got Mail (1998)
userId,,,,,,,,,,,,,,,,,,,,,
5,4.0,4.0,3.5,3.5,4.5,4.0,3.5,4.0,4.0,4.5,...,3.5,4.0,3.5,4.5,4.5,4.0,4.0,4.0,4.0,4.0


In [37]:
# Movies that similar users watched. Remove movies that none of the similar users have watched
sim_users_watched = rating_mt[rating_mt.index.isin(sim_users.index)].dropna(axis=1, how='all')
sim_users_watched

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),13 Going on 30 (2004),2001: A Space Odyssey (1968),21 Grams (2003),25th Hour (2002),28 Days Later (2002),3 Ninjas (1992),...,"X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Zoolander (2001),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,4.5,NaN,...,NaN,4.0,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,NaN
442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,3.5,NaN
500,NaN,4.0,4.5,2.5,4.0,NaN,NaN,NaN,NaN,1.5,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
654,4.0,NaN,NaN,NaN,NaN,4.0,3.5,NaN,4.0,NaN,...,4.0,4.0,4.5,4.0,3.5,4.5,4.5,4.0,3.0,4.5


## Pick the movies from similar users

In [38]:
# Remove the watched movie from the movie list
sim_users_watched.drop(rec_userid_watched.columns,axis=1, inplace=True, errors='ignore')

In [39]:
sim_users_watched

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),13 Going on 30 (2004),2001: A Space Odyssey (1968),21 Grams (2003),25th Hour (2002),3 Ninjas (1992),3 Ninjas Kick Back (1994),...,With Honors (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),You Only Live Twice (1967),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Zoolander (2001),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,3.5,NaN
500,NaN,4.0,4.5,2.5,4.0,NaN,NaN,NaN,1.5,1.5,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN
654,4.0,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,...,4.0,4.0,4.0,4.5,4.0,4.5,4.5,4.0,3.0,4.5


## Recommend movies for user

In [40]:
sim_users

userId
313    0.338180
500    0.336893
292    0.320923
442    0.312862
654    0.310267
Name: 5, dtype: float64

In [41]:
sim_users_watched

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),13 Going on 30 (2004),2001: A Space Odyssey (1968),21 Grams (2003),25th Hour (2002),3 Ninjas (1992),3 Ninjas Kick Back (1994),...,With Honors (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),You Only Live Twice (1967),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Zoolander (2001),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,3.5,NaN
500,NaN,4.0,4.5,2.5,4.0,NaN,NaN,NaN,1.5,1.5,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN
654,4.0,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,...,4.0,4.0,4.0,4.5,4.0,4.5,4.5,4.0,3.0,4.5


In [42]:
# A dictionary to store item scores
item_score = {}
# Loop through items
for i in sim_users_watched.columns:
  # Get the ratings for movie i
  movie_rating = sim_users_watched[i]
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in sim_users.index:
    # If the movie has rating
    if pd.isna(movie_rating[u]) == False:
      # Score is the sum of user similarity score multiply by the movie rating
      score = sim_users[u] * movie_rating[u]
      # Add the score to the total score for the movie so far
      total += score
      # Add 1 to the count
      count +=1
  # Get the average score for the item
  item_score[i] = total / count
  
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
    
# Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)
# Select top m movies
m = 10
ranked_item_score.head(m)

,movie,movie_score
80,Ben-Hur (1959),1.690902
438,Ladder 49 (2004),1.690902
607,Pride and Prejudice (1995),1.684463
456,Life Is Beautiful (La Vita è bella) (1997),1.684463
430,Kiss Kiss Bang Bang (2005),1.684463
863,What Dreams May Come (1998),1.604614
291,Frida (2002),1.604614
292,Fried Green Tomatoes (1991),1.604614
663,"Sandlot, The (1993)",1.604614
834,"Usual Suspects, The (1995)",1.577976




---



---



# Recommend Movies : Item-Based Filtering

## create pivot table for filling cosine similarity

In [43]:
# item-item then switch columns and index from user-user or using transapose
rating_mt_i = rating_small.pivot_table(index='title', columns='userId', values='rating').fillna(0) # calculate cosine similarity the value should not be NaN
print(rating_mt_i.shape)

(8792, 671)


In [44]:
item_similarity_co = cosine_similarity(rating_mt_i)
print(item_similarity_co.shape)
print(type(item_similarity_co))

(8792, 8792)
<class 'numpy.ndarray'>


## Ready to recommend movies to user by item-item

In [45]:
# fill numpy array data to dataframe
item_sim = pd.DataFrame(item_similarity_co, columns=rating_mt_i.T.columns,index=rating_mt_i.T.columns)

In [46]:
item_sim

title,$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (1964),Zulu (2013),[REC] (2007),"\\""Great Performances\""\"" Cats (1998)""",eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
$9.99 (2008),1.000000,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.000000,0.156330,0.108103,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.013899,0.000000,0.058218,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.000000,0.000000,0.413919,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
'Neath the Arizona Skies (1934),0.000000,0.0,1.000000,0.124035,0.0,0.085436,0.000000,0.000000,0.019259,0.000000,...,0.000000,0.000000,0.0,0.164399,0.000000,0.0,0.000000,0.000000,0.000000,0.0
'Round Midnight (1986),0.000000,0.0,0.124035,1.000000,0.0,0.010597,0.143786,0.000000,0.136163,0.000000,...,0.000000,0.000000,0.0,0.020391,0.121567,0.0,0.000000,0.000000,0.000000,0.0
'Salem's Lot (2004),0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.000000,0.000000,0.413919,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loudQUIETloud: A Film About the Pixies (2006),0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.154074,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0
xXx (2002),0.013899,0.0,0.000000,0.000000,0.0,0.000000,0.123940,0.000000,0.144961,0.112392,...,0.161281,0.076029,0.0,0.000000,0.017465,0.0,1.000000,0.152057,0.140222,0.0
xXx: State of the Union (2005),0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.134815,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.152057,1.000000,0.000000,0.0


## Recommend movies for new user by his/her the most favorite movie.

In [47]:
## Example recomed movie for new user that user like 'As Good as It Gets (1997)'

favor_movie = 'As Good as It Gets (1997)'


# Similarity score of the favor movie with all the other movies
picked_movie_similarity_score_n =item_sim[[favor_movie]].reset_index().rename(columns={favor_movie:'similarity_score'})
picked_movie_similarity_score_n =picked_movie_similarity_score_n[picked_movie_similarity_score_n['title']!=favor_movie]
picked_movie_similarity_score_n.nlargest(5,'similarity_score')

# Done

title,title,similarity_score
6337,Rain Man (1988),0.549420
885,Big (1988),0.540848
7779,There's Something About Mary (1998),0.525093
251,Airplane! (1980),0.503680
5063,Men in Black (a.k.a. MIB) (1997),0.492423




---



---



## Recommend movies for exit user by his/her the most favorite movie.

In [48]:
## Example recomed userId 5
rec_userId = 5 

# situation user 5 like 
favor_movie = 'As Good as It Gets (1997)'

## remove userId that picked

# Movies that the target user has watched
item_sim_rec = pd.DataFrame(rating_mt_i[rec_userId].dropna(axis=0, how='all')\
                          .sort_values(ascending=False))\
                          .reset_index()\
                          .rename(columns={rec_userId:'rating'})

item_sim_rec.head(20)

,title,rating
0,Pretty Woman (1990),5.0
1,Grease (1978),5.0
2,Crash (2004),5.0
3,"Sound of Music, The (1965)",5.0
4,"Little Mermaid, The (1989)",5.0
5,Hotel Rwanda (2004),4.5
6,Mrs. Doubtfire (1993),4.5
7,Wedding Crashers (2005),4.5
8,Spider-Man 2 (2004),4.5
9,My Big Fat Greek Wedding (2002),4.5


## Pick the movies from similar items

In [49]:
# Similarity score of the favor movie with all the other movies
movie_sim_favor =item_sim[[favor_movie]].reset_index().rename(columns={favor_movie:'similarity_score'})

# Rank the similarities between the movies user 5 rated and favor movie(As Good as It Gets (1997))
n = 5
userid_watched_sim = pd.merge(left=item_sim_rec, 
                                            right=movie_sim_favor, 
                                            on='title', 
                                            how='inner')\
                                     .sort_values('similarity_score', ascending=False)[:5]
# Take a look at the User 1 watched movies with highest similarity
userid_watched_sim

,title,rating,similarity_score
10,As Good as It Gets (1997),4.5,1.000000
32,Rain Man (1988),4.0,0.549420
6181,Big (1988),0.0,0.540848
20,There's Something About Mary (1998),4.5,0.525093
5984,Airplane! (1980),0.0,0.503680


In [50]:
# Calculate the predicted rating using weighted average of similarity scores and the ratings from user 5
predicted_rating = round(np.average(userid_watched_sim['rating'], 
                                    weights=userid_watched_sim['similarity_score']), 6)

In [51]:
# Calculate the predicted rating using weighted average of similarity scores and the ratings from user 5
predicted_rating = round(np.average(userid_watched_sim['rating'], 
                                    weights=userid_watched_sim['similarity_score']), 4)
print(f'The predicted rating for {favor_movie} by user {rec_userId} is {predicted_rating}' )

The predicted rating for As Good as It Gets (1997) by user 5 is 2.9049


### Remove movies that user has watched

In [52]:
# replcae 0 with nan
rating_mt_i.replace(0,np.nan,inplace=True)

In [53]:
# check result after replace value
j = []
for i in rating_mt_i.columns:
  print(rating_mt_i[i].value_counts())
  j.append(i)
  if len(j) == 3:
    break

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: 1, dtype: int64
3.0    36
4.0    23
5.0    11
2.0     4
1.0     2
Name: 2, dtype: int64
3.0    18
4.0    11
3.5     9
5.0     5
4.5     4
2.5     3
2.0     1
Name: 3, dtype: int64


In [54]:
rating_mt_i[rec_userId]

title
$9.99 (2008)                                    NaN
'Hellboy': The Seeds of Creation (2004)         NaN
'Neath the Arizona Skies (1934)                 NaN
'Round Midnight (1986)                          NaN
'Salem's Lot (2004)                             NaN
                                                 ..
loudQUIETloud: A Film About the Pixies (2006)   NaN
xXx (2002)                                      NaN
xXx: State of the Union (2005)                  NaN
¡Three Amigos! (1986)                           NaN
À nous la liberté (Freedom for Us) (1931)       NaN
Name: 5, Length: 8792, dtype: float64

In [55]:
# Movies that the target user has not watched
userid_unwatched = pd.DataFrame(rating_mt_i[rec_userId].isna()).reset_index()
userid_unwatched

,title,5
0,$9.99 (2008),True
1,'Hellboy': The Seeds of Creation (2004),True
2,'Neath the Arizona Skies (1934),True
3,'Round Midnight (1986),True
4,'Salem's Lot (2004),True
...,...,...
8787,loudQUIETloud: A Film About the Pixies (2006),True
8788,xXx (2002),True
8789,xXx: State of the Union (2005),True
8790,¡Three Amigos! (1986),True


In [56]:
userid_unwatched = userid_unwatched[userid_unwatched[rec_userId]==True]['title'].values.tolist() # True mean the data has missing value

In [57]:
# Movies that the target user has watched
userid_watched = pd.DataFrame(rating_mt_i[rec_userId].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={rec_userId:'rating'})

In [58]:
userid_watched

,title,rating
0,Crash (2004),5.0
1,"Little Mermaid, The (1989)",5.0
2,Grease (1978),5.0
3,Pretty Woman (1990),5.0
4,"Sound of Music, The (1965)",5.0
...,...,...
95,Harry Potter and the Chamber of Secrets (2002),3.0
96,Lost in Translation (2003),2.5
97,"Godfather, The (1972)",2.5
98,"Godfather: Part II, The (1974)",2.5


In [59]:
picked_movie_similarity_score = item_sim[[favor_movie]].reset_index().rename(columns={favor_movie:'similarity_score'})

In [60]:
picked_movie_similarity_score

title,title,similarity_score
0,$9.99 (2008),0.023892
1,'Hellboy': The Seeds of Creation (2004),0.000000
2,'Neath the Arizona Skies (1934),0.000000
3,'Round Midnight (1986),0.103743
4,'Salem's Lot (2004),0.000000
...,...,...
8787,loudQUIETloud: A Film About the Pixies (2006),0.000000
8788,xXx (2002),0.119233
8789,xXx: State of the Union (2005),0.000000
8790,¡Three Amigos! (1986),0.322187


In [61]:
item_sim.isnull().sum().sum()

0

In [62]:
userid_watched_sim

,title,rating,similarity_score
10,As Good as It Gets (1997),4.5,1.000000
32,Rain Man (1988),4.0,0.549420
6181,Big (1988),0.0,0.540848
20,There's Something About Mary (1998),4.5,0.525093
5984,Airplane! (1980),0.0,0.503680


In [63]:
userid_watched_sim['similarity_score'].sum()

3.119040886877519

In [64]:
userid_watched['rating']

0     5.0
1     5.0
2     5.0
3     5.0
4     5.0
     ... 
95    3.0
96    2.5
97    2.5
98    2.5
99    1.5
Name: rating, Length: 100, dtype: float64

### Recommend movies for exit user

In [65]:
# Dictionary to save the unwatched movie and predicted rating pair
n = 10
rating_prediction ={}  
number_of_similar_items = n
number_of_reccomend = n
# Loop through unwatched movies          
for picked_movie in userid_unwatched: 
  # Calculate the similarity score of the picked movie iwth other movies
  movie_sim_favor = item_sim[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
  # Rank the similarities between the picked user watched movie and the picked unwatched movie.
  userid_watched_sim = pd.merge(left=userid_watched, 
                                              right=movie_sim_favor, 
                                              on='title', 
                                              how='inner').\
                                              sort_values('similarity_score', ascending=False)[:number_of_similar_items]
  weight = userid_watched_sim['similarity_score']
  # if weight == 0 ,The calculation will fail.
  if weight.sum() != 0 :
    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 5
    predicted_rating = round(np.average(userid_watched_sim['rating'], 
                                        weights=userid_watched_sim['similarity_score']), 4)
    # Save the predicted rating in the dictionary
    # print(picked_movie,predicted_rating)
    rating_prediction[picked_movie] = predicted_rating

In [66]:
ranked_item_score_i = pd.DataFrame(data=rating_prediction.items(),columns=['title','rating'])
ranked_item_score_i.sort_values(by='rating',ascending=False,inplace=True)
ranked_item_score_i.head(number_of_reccomend)

,title,rating
8409,Why Do Fools Fall In Love? (1998),4.5000
8524,Woo (1998),4.5000
1610,"Clique, The (2008)",4.5000
5111,Moll Flanders (1996),4.4637
428,Angels in the Outfield (1994),4.4193
2583,Fame (1980),4.4080
3224,"Great Mouse Detective, The (1986)",4.4018
669,"Band Wagon, The (1953)",4.4008
6040,Porky's Revenge (1985),4.3987
3767,I Was a Male War Bride (1949),4.3962




---



---



In [67]:
# Done